In [18]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

# LOAD WORD2VEC MODEL TRAIN FROM PREVIOUS TASK

In [19]:
model = Word2Vec.load("./ductrung239-projects-quang-song-to-vec-15-home/song2vec.model")

## Some pre-processing steps for normalization text

In [20]:
import pandas as pd

import random

import re
import string
import os

regex = re.compile('[%s]' % re.escape(string.punctuation))
def cleaning_text(doc):
    
    # convert to lower case
    doc = doc.lower()

    # remove spacespace
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()
    doc = doc.replace(" ", "_")
    return doc


def read_data():
    df = pd.read_csv(
        "./songs_dataset_105k.csv")
    df.drop(columns=['ID', 'Md5 ', 'album_link', 'song_link', "Site"], inplace=True)

    values = {'Singer': 'Singer unknown', 
            'Song': "Song unknown", 
            'Author': "Author unknown", 
            'Album': "Album unknown", 
            'Genre': "Genre unknown"}

    df.fillna(value=values, inplace=True)


    df['Song'] = df['Song'].astype('str')
    df['Singer'] = df['Singer'].astype('str')
    df['Author'] = df['Author'].astype('str')
    df['Album'] = df['Album'].astype('str')
    df['Genre'] = df['Genre'].astype('str')

    df['Song'] = df['Song'].apply(cleaning_text)
    df['Singer'] = df['Singer'].apply(cleaning_text)
    df['Author'] = df['Author'].apply(cleaning_text)
    df['Album'] = df['Album'].apply(cleaning_text)
    df['Genre'] = df['Genre'].apply(cleaning_text)

    return df 

# Load pre-train dataset
df = read_data()

In [21]:
dict_category = dict()
missing_item = 0
for col in df.columns:
    value_list = []
    value_vectors = []
    for value in df[col].unique():
        try:
            value_vectors.append(model.wv.get_vector(value))
            value_list.append(value)
        except:
            missing_item += 1
    dict_category[col] = {
        "value_list": value_list,
        "value_vectors": value_vectors
    }
print("Missing item: ", missing_item)
print(dict_category.keys())

Missing item:  3
dict_keys(['Song', 'Singer', 'Author', 'Album', 'Genre'])


## Build tress from each categories

In [22]:
from sklearn.neighbors import KDTree
import numpy as np
dict_of_tree = dict()

for key in dict_category.keys():
    print("Key: ", key)
    tree = KDTree(np.array(dict_category[key]['value_vectors']), leaf_size=30, metric='euclidean')
    dict_of_tree[key] = tree

Key:  Song
Key:  Singer
Key:  Author
Key:  Album
Key:  Genre


## Some test cases

### Find top 10 songs  similar with "hạ_trắng"

In [23]:
dist, ind = dict_of_tree['Song'].query(model.wv.get_vector("hạ_trắng").reshape(1, -1), k=10)
print("10 songs most similar with 'hạ_trắng' ")
for i in ind[0]:
    print(dict_category['Song']['value_list'][i])

10 songs most similar with 'hạ_trắng' 
hạ_trắng
tuổi_đá_buồn
diễm_xưa
cát_bụi
nắng_thủy_tinh
một_cõi_đi_về
đêm_thấy_ta_là_thác_đổ
ru_em_từng_ngón_xuân_nồng
như_cánh_vạc_bay
hạ_trắng_(beat)


### Find top 10 singers  similar with "trịnh_công_sơn"

In [24]:
dist, ind = dict_of_tree['Singer'].query(model.wv.get_vector("trịnh_công_sơn").reshape(1, -1), k=15)
print("Top 10 Singer most similar with 'trịnh_công_sơn' ")
for i in ind[0]:
    print(dict_category['Singer']['value_list'][i])

Top 10 Singer most similar with 'trịnh_công_sơn' 
trịnh_công_sơn
hoàng_lân
trâm_anh
nguyễn_hữu_thái_hòa
công_luân
thế_duật
phương_ngọc
chính_duật
ngô_quang_vinh
thúy_hà_tú
kim_ánh
thủy_tiên_(trịnh)
bích_ngà
giang_trang
nguyệt_ca


## Check data in dataset

In [25]:
# find all row have singer is "nguyệt_ca"
df[df.Singer == 'nguyệt_ca']

,Song,Singer,Author,Album,Genre
8,bốn_mùa_thay_lá,nguyệt_ca,trịnh_công_sơn,bốn_mùa_thay_lá_-,nhạc_trịnh
22579,cũng_sẽ_chìm_trôi,nguyệt_ca,trịnh_công_sơn,môi_hồng_đào_-,nhạc_trịnh
22593,hãy_yêu_nhau_đi,nguyệt_ca,trịnh_công_sơn,mẹ_-_cánh_chim_cô_đơn_-,nhạc_trịnh
94593,hãy_yêu_nhau_đi,nguyệt_ca,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh
94613,lời_mẹ_ru,nguyệt_ca,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh
94616,cũng_sẽ_chìm_trôi,nguyệt_ca,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh
94618,mưa_hồng,nguyệt_ca,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh
94620,níu_tay_nghìn_trùng,nguyệt_ca,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh
94622,đóa_hoa_vô_thường,nguyệt_ca,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh
94623,cát_bụi,nguyệt_ca,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh


In [26]:
# find all row have singer is "nguyễn_hữu_thái_hòa"
df[df.Singer == 'nguyễn_hữu_thái_hòa']

,Song,Singer,Author,Album,Genre
22578,cũng_sẽ_chìm_trôi,nguyễn_hữu_thái_hòa,trịnh_công_sơn,môi_hồng_đào_-,nhạc_trịnh
22603,hãy_yêu_nhau_đi,nguyễn_hữu_thái_hòa,trịnh_công_sơn,mẹ_-_cánh_chim_cô_đơn_-,nhạc_trịnh
94581,hãy_yêu_nhau_đi,nguyễn_hữu_thái_hòa,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh
94619,cũng_sẽ_chìm_trôi,nguyễn_hữu_thái_hòa,trịnh_công_sơn,những_bài_hát_hay_nhất_của_nguyệt_ca,nhạc_trịnh


In [27]:
# find all row have singer is "khánh_ly"
df[df.Singer == 'khánh_ly']

,Song,Singer,Author,Album,Genre
1765,giáo_đường_im_bóng,khánh_ly,nguyễn_thiện_tơ,những_bài_hát_giáng_sinh_hải_ngoại_hay_nhất,trữ_tình
1842,chiều_bên_giáo_đường,khánh_ly,lê_trọng_nguyễn,chuyện_mùa_giáng_sinh_(tuyển_tập_trữ_tình),trữ_tình
1851,lời_buồn_thánh,khánh_ly,trịnh_công_sơn,chuyện_mùa_giáng_sinh_(tuyển_tập_trữ_tình),nhạc_trịnh
1935,quỳnh_hương,khánh_ly,trịnh_công_sơn,chuyện_đêm_trăng_(tuyển_tập_trữ_tình),nhạc_trịnh
2010,tôi_đang_lắng_nghe,khánh_ly,trịnh_công_sơn,tình_khúc_trịnh_công_sơn_-_vol_5_-,nhạc_trịnh
2027,liên_khúc_trịnh_công_sơn,khánh_ly,trịnh_công_sơn,niệm_khúc_cuối_-_top_hits_31_-,nhạc_trịnh
2140,vườn_thu,khánh_ly,văn_thủy,bài_tango_cuối_cùng_-,trữ_tình
2165,mộng_lành,khánh_ly,hoàng_trọng,bài_tango_cho_em_-,trữ_tình
2339,gởi_em_hành_lý,khánh_ly,trầm_tử_thiêng,kinh_khổ_-,trữ_tình
2365,chìm_dưới_cơn_mưa,khánh_ly,trịnh_công_sơn,im_lặng_thở_dài_-,nhạc_trịnh


In [28]:
dist, ind = dict_of_tree['Song'].query(model.wv.get_vector("trịnh_công_sơn").reshape(1, -1), k=10)
print("Top 10 songs most similar with 'trịnh_công_sơn' ")
for i in ind[0]:
    print(dict_category['Song']['value_list'][i])

Top 10 songs most similar with 'trịnh_công_sơn' 
tình_nhớ
mưa_hồng
tuổi_đá_buồn
hoa_vàng_mấy_độ
như_một_lời_chia_tay
chiều_một_mình_qua_phố
diễm_xưa
biển_nghìn_thu_ở_lại
cát_bụi
đêm_thấy_ta_là_thác_đổ


## Find all song similar with "trinh_cong_son" and "pham_duy"

In [29]:
trinh_cong_son_vector = model.wv.get_vector("trịnh_công_sơn")
pham_duy_vector = model.wv.get_vector("phạm_duy")

In [30]:
mean_of_vectors = (trinh_cong_son_vector + pham_duy_vector)/2

In [31]:
dist, ind = dict_of_tree['Song'].query(mean_of_vectors.reshape(1, -1), k=10)
for i in ind[0]:
    print(dict_category['Song']['value_list'][i])

nha_trang_ngày_về
nghìn_năm_vẫn_không_quên
kiếp_nào_có_yêu_nhau
giàn_thiên_lý_đã_xa
đưa_em_tìm_động_hoa_vàng
mẹ_hoá_đá
nước_mắt_mùa_thu
rồi_đây_anh_sẽ_đưa_em_về_nhà
giữ_gìn_tình_anh
biển_nghìn_thu_ở_lại


In [52]:
df[df.Singer == "lê_nghĩa"]

,Song,Singer,Author,Album,Genre
59432,được_làm_người,lê_nghĩa,various_artists_1,nhạc_kịch_tấm_cám_-,nhạc_trẻ


In [51]:
df[df.Song == "cô_đơn_một_vì_sao"]

,Song,Singer,Author,Album,Genre
3930,cô_đơn_một_vì_sao,ưng_hoàng_phúc,ưng_hoàng_phúc,những_bài_hát_hay_nhất_của_ưng_hoàng_phúc,nhạc_trẻ


In [50]:
dist, ind = dict_of_tree['Singer'].query(model.wv.get_vector("cô_đơn_một_vì_sao").reshape(1, -1), k=10)
print("Top 10 songs most similar with 'cô_đơn_một_vì_sao' ")
for i in ind[0]:
    print(dict_category['Singer']['value_list'][i])

Top 10 songs most similar with 'cô_đơn_một_vì_sao' 
lê_nghĩa
ưng_hoàng_phúc
chenco
trúc_nhi
nguyễn_phong
pittu_quyên
phong_windy
three_b
thảo_vân
jenny
